# packages

In [11]:
#import subprocess
#import sys

#def install(package):
#    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

#install("pandas")
#install("numpy")
#install("matplotlib")
#install("openpyxl")

# imports

In [12]:
import pandas as pd
import numpy as np
import json
import csv
import os

In [13]:
heads = ['ID', 'Title', 'model processing technique', 'source', 'source category', 'source sub-category 1', 'source sub-category 2', 'source model target category', 'target', 'target category', 'target sub-category 1', 'target sub-category 2', 'target model target category', 'Technological Readiness Level', 'system lifecycle phase', 'twinning target', 'twin lifecycle phase', 'use case domain','Use Case']


# Read Extraction Data from CSV

In [14]:
data = pd.read_csv(filepath_or_buffer="./data/04 extraction consolidation results.csv",sep=";")
titles = data.drop_duplicates(subset=["ID"])
titles = titles["Title"]
data.head()

,ID,Title,model processing technique,source,source category,source sub-category 1,source sub-category 2,source model target category,target,target category,target sub-category 1,target sub-category 2,target model target category,Technological Readiness Level,system lifecycle phase,twinning target,twin lifecycle phase,use case domain,Use Case
0,1,A digital twin-driven hybrid approach for the ...,Model Interpretation,"hybrid model (physical, geometric, behavior, r...",behavior + structure model,data,NaN,PT,performance degredation measurement algorithm,Software,data processing,NaN,NaN,Evaluation Research,Design,Individual Systems,As-Operated,C - Manufacturing,Transmission of CNC Machine
1,1,A digital twin-driven hybrid approach for the ...,Model Interpretation,runtime data,structural model,data,NaN,PT,simulation engine,Software,simulation software,NaN,NaN,NaN,Design,NaN,NaN,NaN,NaN
2,2,A framework for Model-Driven Engineering of re...,Code Generation,SysML: BDD,structural model,data,NaN,PT,Knowledge Base of software digital twins (Meta...,Software,data storage,NaN,NaN,Validation Research,Design,Individual Systems,As-Designed,H - Transport,limited traffic zone access control
3,2,A framework for Model-Driven Engineering of re...,Model Interpretation,Fault Trees,behavior model,discrete behavior,NaN,PT,FaultTreeAnalyzer (metamodel proposed in Figur...,Software,control,NaN,NaN,NaN,Design,NaN,NaN,NaN,NaN
4,3,A Methodology for Digital Twin Modeling and De...,Code Generation,AutomationML models,structural model,data,NaN,PT,DT runtime memory,software,other software,NaN,NaN,Validation Research,Operate,Systems of Systems,As-Operated,C - Manufacturing,refinery automationsystem with four valves


# create JSON from CSV

In [15]:
keys = [ 'Technological Readiness Level',  'twinning target', 'twin lifecycle phase', 'use case domain', 'Use Case']
mde_keys = ['model processing technique', 'system lifecycle phase', 'source', 'source category', 'source sub-category 1', 'source sub-category 2','source model target category', 'target', 'target category', 'target sub-category 1', 'target sub-category 2', 'target model target category',]

In [16]:
result = {}


In [17]:
data.reset_index()
for title in titles:
    rownum = 0
    current_rows = data[data["Title"] == title]
    current_row = current_rows.iloc[0]
    result[title] = {}
    for key in keys:
        if current_row[key] is None or pd.isna(current_row[key]):
            result[title][key] = None
        else:
            result[title][key] = current_row[key]
            if key == "use case domain" and current_row[key] == "Accommodation":
                print(title)
    result[title]["modelTrafos"] = []
    for index, technique_in in current_rows.iterrows():
        technique_out = {}
        for key in mde_keys:
            if technique_in[key] is None or pd.isna(technique_in[key]):
                technique_out[key] = None 
            else:
                technique_out[key] = technique_in[key]
        result[title]["modelTrafos"].append(technique_out)

In [18]:
json_object = json.dumps(result, indent = 4) 

with open("./data/data_consolidated_semistructured.json", "w+") as outfile:
    outfile.write(json_object)

# create list of venues

In [19]:
metadata = pd.read_csv("./data/00 Search Results.csv", sep=";", encoding="latin-1")
metadata = metadata.drop_duplicates(subset=['Title'])
data_merged = data.drop_duplicates(subset=["ID"])
print(len(data_merged))
data_merged = data_merged.merge(metadata, how='left', on='Title')
print(len(data_merged))
print(data_merged.columns)

64
64
Index(['ID', 'Title', 'model processing technique', 'source',
       'source category', 'source sub-category 1', 'source sub-category 2',
       'source model target category', 'target', 'target category',
       'target sub-category 1', 'target sub-category 2',
       'target model target category', 'Technological Readiness Level',
       'system lifecycle phase', 'twinning target', 'twin lifecycle phase',
       'use case domain', 'Use Case', 'Publication Title', 'Authors',
       'Publication Year', 'Publisher'],
      dtype='object')


In [20]:
data_merged.to_csv("test.csv", sep=";")